In [11]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
load_dotenv()
from pymongo import MongoClient
import sys
sys.path.append('../src')
import mongofun as mon

In [9]:
keylast = os.getenv("keylast")

In [10]:
client = MongoClient("localhost:27017")
lastusers = client.get_database("lastusers")

 ```python
def mongouser(lastuser):
    if lastuser.lower() not in lastusers.list_collection_names():
        lastusers.create_collection(lastuser.lower())
        coluser = lastusers.get_collection(f"{lastuser.lower()}")

        url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&api_key={keylast}&format=json'
        req = requests.get(url).json()['recenttracks']
        scrobs = req['@attr']['total']
        pages = int(req['@attr']['totalPages'])
        print(f'recovering {scrobs} scrobbles in {pages} pages')


        for i in range(1,pages+1):
            page = i
            url_t = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&page={page}&api_key={keylast}&format=json'
            req_t = requests.get(url_t).json()['recenttracks']['track']
            for r in req_t:
                if '@attr' not in r.keys():
                    coluser.insert_one(r)
                elif '@attr' in r.keys() and i ==1:
                    art_play = r['artist']['#text']
                    tit_play = r['name']
                    print (f'Now playing: {art_play.capitalize()} - {tit_play.capitalize()}')

            if i%5==0:
                print(f'pag {i} done')
            elif i == pages:
                print('all done, bitches')
    else:
       
        coluser = lastusers.get_collection(f"{lastuser.lower()}")
        uts_num = utsmongo(coluser) + 1
        url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&from={uts_num}&api_key={keylast}&format=json'
        req = requests.get(url).json()['recenttracks']

         
        if req['@attr']['totalPages'] == '0':
            if len(req['track']) == 0:            
                return 'no hay nada que insertar'
            
            else:
                if '@attr' in req['track'].keys():                    
                    art_play = req['track']['artist']['#text']
                    tit_play = req['track']['name']
                    print('nada que insertar')
                    return f'Now playing: {art_play.capitalize()} - {tit_play.capitalize()}'


        elif req['@attr']['totalPages'] != '0':   
            scrobs = req['@attr']['total']
            pages = int(req['@attr']['totalPages'])
            print(f'recovering {scrobs} scrobbles in {pages} pages')

            for i in range(1,pages+1):
                page = i
                url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&from={uts_num}&page={page}&api_key={keylast}&format=json'
                req = requests.get(url).json()['recenttracks']


                for r in req['track']:
                    if '@attr' in r.keys() and i ==1:
                        art_play = r['artist']['#text']
                        tit_play = r['name']
                        print (f'Now playing: {art_play.capitalize()} - {tit_play.capitalize()}')
                        
                    elif '@attr' not in r.keys():
                        coluser.insert_one(r)
                if i%5==0:
                    print(f'pag {i} done')
                elif i == pages:
                    print('all done, bitches')
        else:
            print('no insertamos nada porque nada hay que insertar')

```

In [4]:
lastuser = 'avekaesar'

In [6]:
url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&api_key={keylast}&format=json'
req = requests.get(url).json()['recenttracks']
scrobs = req['@attr']['total']
pages = int(req['@attr']['totalPages'])
print(f'recovering {scrobs} scrobbles in {pages} pages')

recovering 88860 scrobbles in 89 pages


In [12]:
coluser = lastusers.get_collection(f"{lastuser.lower()}")
uts_num = mon.utsmongo(coluser) + 1
url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&from={uts_num}&api_key={keylast}&format=json'
req = requests.get(url).json()['recenttracks']

In [13]:
req

{'track': [],
 '@attr': {'user': 'AveKaesar',
  'totalPages': '0',
  'page': '1',
  'perPage': '1000',
  'total': '0'}}